##  Scraping dataset from Wikipedia

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from this url : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [7]:
soup = BeautifulSoup(source, 'xml')

In [8]:
table=soup.find('table')

In [9]:
column_names = ['Postalcode','Borough','Neighborhood'] #df has 3 columns
df = pd.DataFrame(columns = column_names)

In [14]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [15]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


remove rows where Borough is 'Not assigned'

In [16]:
df=df[df['Borough']!='Not assigned']

In [17]:
df[df['Neighborhood']=='Not assigned']=df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [18]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [19]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [20]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [21]:
df_merge.drop_duplicates(inplace=True)

In [22]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [23]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods, Parkwoods"
3,M4A,North York,"Victoria Village, Victoria Village, Victoria V..."
6,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront"
9,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
15,M7A,Downtown Toronto,"Queen's Park, Queen's Park, Queen's Park"


In [24]:
df_merge.shape

(103, 3)

In [25]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [26]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [27]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [31]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [33]:
geo_merged

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"
5,M1J,43.744734,-79.239476,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
6,M1K,43.727929,-79.262029,Scarborough,"East Birchmount Park, Ionview, Kennedy Park, E..."
7,M1L,43.711112,-79.284577,Scarborough,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol..."
8,M1M,43.716316,-79.239476,Scarborough,"Cliffcrest, Cliffside, Scarborough Village Wes..."
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


In [38]:
geo_data = geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [39]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae",43.773136,-79.239476
